#### Building ecole

* [configure with cmake](../ecole/docs/contributing.rst#L93) and [build](../ecole/docs/contributing.rst#L112)
* before building, do not forget to update the [CMakeLists.txt](../ecole/libecole/CMakeLists.txt) by including the new `*.cpp` files in the `add_library` section
* **DO NOT** forget to prefix jupyter or python with `PYTHONPATH="${PWD}/build/cmake/python/ecole"` when developing
* ensure no warnings and use `pybind11`
* these [docs](./ecole/docs/create-environment.rst) are very useful

```bash
# conda update -n base -c defaults conda
# conda deactivate && conda env remove -n learn2branch

conda create -n learn2branch "python>3.9" pip setuptools \
  numpy "scipy>1.7" matplotlib scikit-learn notebook ipywidgets \
  scikit-build cython cmake pybind11 clang-tools graphviz \
  "pytorch::pytorch==1.12.1" networkx "conda-forge::pyscipopt"

# developing
conda activate learn2branch
cd ecole

# configure with CMake
./dev/run.sh configure -D ECOLE_DEVELOPER=ON

# rebuild
clear && ./dev/run.sh build-lib -- build-py

export PYTHONPATH="${PWD}/build/cmake/python/ecole"
echo $PYTHONPATH

PYTHONPATH=$PYTHONPATH jupyter notebook --no-browser --prot=8889
```

#### Inversion of control by Ecole

Ecole implements control inversion with the help of a thread-based [coroutine](./ecole/libecole/include/utility/coroutine.hpp) and SCIP's [obj*](./scip-8.0.1/src/objscip/) plug-in interface.

Considering branching rules as an example, Ecole [implements](./ecole/libecole/src/scip/scimpl.cpp#L70-169) a C++ level interface [ObjBranchrule](./scip-8.0.1/src/objscip/objbranchrule.h) for a special branching rule and registers it through [SCIPincludeObjBranchrule](./scip-8.0.1/src/objscip/objbranchrule.cpp#229)
in [include_reverse_callback](./ecole/libecole/src/scip/scimpl.cpp#L171) called by [solve_iter()](./ecole/libecole/src/scip/scimpl.cpp#L389) in SCIP's fresh coro thread.

Ecole's special object lives in __SCIP's thread__ and intercepts it's indirect calls to `scip_exec*` methods. The call chain from [SCIPsolve](./scip-8.0.1/src/scip/scip_solve.c#L2613)
1. [SCIPsolveCIP](./scip-8.0.1/src/scip/scip_solve.c#L2745)
1. [solveNode](./scip-8.0.1/src/scip/solve.c#L4982)
2. [SCIPbranchExecLP](./scip-8.0.1/src/scip/solve.c#L4420)
3. [SCIPbranchruleExecLPSol](./scip-8.0.1/src/scip/solve.c#L2560)
4. [branchrule->branchexeclp()](./scip-8.0.1/src/scip/solve.c#L1581)
6. [SCIP_Branchrule.branchexeclp](./scip-8.0.1/src/scip/struct_branch.h#L91) is set to `scip_execlp` method by [SCIPincludeObjBranchrule](./scip-8.0.1/src/objscip/objbranchrule.cpp#229)

Ecole then routes each call to the coro executor [std::weak_ptr\<Executor\> m_weak_executor](./ecole/libecole/src/scip/scimpl.cpp#L161), which then [yields](./ecole/libecole/src/scip/scimpl.cpp#L64) (suspends SCIP's thread, transfers data and control) to ecole's thread, which is resumed at a [wait](./ecole/libecole/src/scip/scimpl.cpp#L400). Either from [solve_iter](./ecole/libecole/src/scip/scimpl.cpp#L386) or [solve_iter_continue](./ecole/libecole/src/scip/scimpl.cpp#L398) the message from the coro ends up processed by the current [dynamics](./ecole/libecole/src/dynamics/branching.cpp#L43) and the control finally returns to python. Afterwards, an eventual call to [step_dynamics](./ecole/libecole/src/dynamics/branching.cpp#L75) passes through to [solve_iter_continue](./ecole/libecole/src/scip/scimpl.cpp#L398) which suspends the Ecole's thread, and [resumes](./ecole/libecole/src/scip/scimpl.cpp#L399) the SCIP's thread at the end of its current `yield`.

This interaction is very similar to python's generator-as-a-coroutine interface
* `a = yield b  <<-->> b = .send(a)`

The code below ilustrates the cooperation between ecole and scip
```python
# types in `scmipl.cpp`
DynamicConstructor = callable
DynamicCall = object
TerminalCall = object
SCIP_RESULT = int
Action = object

class ecole:
    """Hopefully this illustrates the interaction.

    Lines are as of commit 2326b70 in the `nodesel` branch
    """

    m_controller: iter
    n_iter: int
    Observation = (object, bool, int)

    def scip(
        self,
        ctor: DynamicConstructor,
    ) -> DynamicCall:
        """./ecole/libecole/src/scip/scimpl.cpp#L391"""
        # ReverseBranchrule calls SCIPincludeObjBranchrule
        n_total_iter = ctor()

        # SCIP_STAGE_SOLVING
        n_iter = 0
        while n_iter < n_total_iter:
            # SCIP can be modified while its thread is suspended
            result = yield DynamicCall
            n_iter += 1
            if result is not SCIP_RESULT:
                break

        # SCIP_STAGE_SOLVED
        return TerminalCall
    
    def affect_scip(self, act: Action) -> SCIP_RESULT:
        """./ecole/libecole/src/dynamics/branching.cpp#L70"""
        # implement `act` by calling `SCIP*`, then return
        #  the SCIP_RESULT (SCIP_DIDNOTRUN, SCIP_SUCCESS)

        # return self.scip.branch_var(act)
        return SCIP_RESULT
    
    def solve_iter(
        self,
        ctor: DynamicConstructor,
    ) -> DynamicCall:
        """./ecole/libecole/src/scip/scimpl.cpp#L386"""
        # init-wait
        self.m_controller = self.scip(ctor)
        request = self.m_controller.send(None)

        assert request is DynamicCall
        return request

    def solve_iter_continue(
        self,
        result: SCIP_RESULT,
    ) -> DynamicCall:
        """./ecole/libecole/src/scip/scimpl.cpp#L398"""
        # resume-wait
        request = self.m_controller.send(result)

        assert request is DynamicCall
        return request

    def reset(self, n_total: int) -> Observation:
        """./ecole/libecole/src/dynamics/branching.cpp#L51"""
        try:
            self.n_iter = 0
            return (
                self.solve_iter(lambda: n_total),
                False,
                self.n_iter,
            )

        except StopIteration as e:
            """./ecole/libecole/src/dynamics/branching.cpp#L36"""
            assert e.value is TerminalCall
            return (None, True, self.n_iter)

    def step(self, act: Action) -> Observation:
        """./ecole/libecole/src/dynamics/branching.cpp#L56"""
        try:
            self.n_iter += 1
            result = self.affect_scip(act)
            return (
                self.solve_iter_continue(result),
                False,
                self.n_iter,
            )

        except StopIteration as e:
            """./ecole/libecole/src/dynamics/branching.cpp#L36"""
            assert e.value is TerminalCall
            return (None, True, self.n_iter)


# launch ecole
env = ecole()

obs, fin, nfo = env.reset(3)
while not fin:
    print(obs, fin, nfo)
    obs, fin, nfo = env.step(Action)
print(obs, fin, nfo)
```

* is this code worth a thousand words?

#### nodesel

the [Nodesel](./ecole/libecole/src/dynamics/nodesel.cpp) scip dynamics is almost a carbon copy of the branchrule interceptor.

The main issue is that the next node `SCIP_NODE *nextnode`, when being selected form the global prio-queue with [SCIPnodeselSelect](./scip-8.0.1/src/scip/solve.c#L5159) of the active `nodesel`, appears to be in an unprocessed state
* in fact the `SCIP_NODE *focusnode` is actually set in the cutoff sub-loop [#L4914-4953](./scip-8.0.1/src/scip/solve.c#L4933)
* the node is processed by [SCIPnodeFocus](./src/scip/solve.c#L4943)
* [SCIPnodeFocus](./scip-8.0.1/src/scip/tree.c#L4298-4756) cuts off, checks infeasibility and maintain the bnb tree (based on a cursory reading)
* only then it calls [solveNode](./scip-8.0.1/src/scip/solve.c#L4985), which adds cuts, domain tightening and etc.

##### Must implement

The nodesel interface [SCIP_Nodesel](./scip-8.0.1/src/scip/type_nodesel.h#L101-131):
- [x] [ObjNodesel->scip_select(..., SCIP_NODE** selnode)](./scip-8.0.1/src/objscip/objnodesel.h#L136-140) put the the selected `SCIP_NODE*` into `*selnode`
- [x] [ObjNodesel->scip_comp(..., SCIP_NODE* n1, SCIP_NODE* n2)](./scip-8.0.1/src/objscip/objnodesel.h#L142-146) return `0` if `n1` and `n2` are equally good, `> 0` if `n2` is worse than `n1`, `< 0` otherwise
  - we may leave this as a trivial function returning 0, but it is better to compare the nodes by their lower bound (SCIP transparently transforms a porblem into minimization)
- Apparently different nodesels manage and resort the same `.leaves` pq
 - a good example of a `nodesel` is [nodesel_uct](./scip-8.0.1/src/scip/nodesel_uct.h#L189-242), which computes the UCT score of the leave nodes during `nodeselSelectUct` based on
$$
\mathrm{uct}_v
    = \underbrace{
        \frac{
            \mathrm{lp}_\circ - \mathrm{lp}_v
        }{\max\{1, \min\{
            \lvert \mathrm{lp}_\circ \rvert,
            \lvert \mathrm{lp}_v \rvert
        \}\}}
    }_{\text{score of node } v}
    + C \frac{n_{\pi_v}}{1 + n_v}
    \,. $$
  - SCIP also has a Bandit plugin support for adaptive large neighbourhood search [heurExecAlns](./scip-8.0.1/src/scip/heur_alns.c#L2294)
$$
\mathrm{ucb}_a
    = \hat{\mu}_a
    + \alpha \sqrt{\frac{\log{(1 + n})}{n_a}}
\,. $$

#### nodesel proper

- [x] implement the number-to-node mapping [nodesel.cpp](../ecole/libecole/src/dynamics/nodesel.cpp)
  - the action is the number which has to be manually mapped back to an open node, unlike branchrule, which returns the natural variabls indices
  - we could communicate `SCIP_NODE*` [py::capsules](https://docs.python.org/3/c-api/capsule.html) but that would break the separation of the python-related code from the pure cpp-core of `libecole`
  - use `scip::call(SCIPgetOpenNodesData, ...)` to get the open leaf, child and sibling nodes
- [x] decalre the nodesel interceptor class [nodesel.hpp](../ecole/libecole/include/ecole/dynamics/nodesel.hpp)
- [x] define the nodesel constructor and dynamic calls [callback.hpp](../ecole/libecole/include/ecole/scip/callback.hpp)
- [ ] 

#### Alternatives to Ecole

Btw, there might be a way to implement nodesel and branchrule through `pyscipopt`. Especially, since they expose the node objects, and branchrule objects as well. Although it is possible to collect features using the bare `pyscipopt` interface (`pyscipopt>=4.0`), it appears that we can access the LP data of the __focus node__ only, and all open nodes on their own do not have methods to retrieve any LP, because they have not been visited/processed. Possibly, this is how it is for reasons of memory conservation, especially since the forntier of grows super-linearly.

```python
import pyscipopt as scip

# Base class of the Nodesel Plugin
class NodeselDynamics(scip.Nodesel):
    # ./src/scip/struct_nodesel.h
    # https://github.com/scipopt/PySCIPOpt/blob/master/src/pyscipopt/nodesel.pxi
    # https://github.com/ds4dm/learn2branch/blob/master/02_generate_dataset.py
    def __init__(self) -> None:
        # No need to pass a scip.Model here, since  `m.includeNodesel`
        #  already inits the `self.model` to the current model
        pass

    def nodeinit(self):
        pass

    def nodeselect(self) -> dict[str, scip.Node]:
        # https://github.com/scipopt/PySCIPOpt/blob/master/src/pyscipopt/nodesel.pxi#L85-L92
        return {"selnode": None}

    def nodecomp(self, n1: scip.Node, n2: scip.Node) -> int:
        return 0

m = scip.Model()

# ...

nodesel = NodeselDynamics()
m.includeNodesel(
    nodesel=nodesel,
    name="",
    desc="",
    stdpriority=666_666,
    memsavepriority=666_666,
)
# we can add other interceptors here and we will get
#  called at appropriate times.
```

<br>

In [ ]:
import ecole as ec

c = ec.scip.callback.NodeselCall()

In [ ]:
import ecole as ec
from pyscipopt import scip

class NodeselEnv(ec.environment.Environment):
    __Dynamics__ = ec.dynamics.NodeselDynamics
    

it = ec.instance.SetCoverGenerator()


env = NodeselEnv(
    # observation_function=ec.observation.NodeBipartite(),
    scip_params= {
        "timing/clocktype": 2,  # XXX Wall clock time
        "limits/time": 3600,
        #     "limits/gap": 1e-2,
        #     "limits/nodes": 10,
        # deactivate presolving
        "presolving/maxrounds": 0,
        "separating/maxrounds": 0,
        "presolving/maxrestarts": 0,
        # sum score function
        "branching/scorefunc": "s",
        #     "branching/scorefac": 0.5,
        # SCIP-s default FSB heuristic is an advanced version of SB and non-idempotent
        #  meaning that it may silently alter the state of the search tree
        #     "branching/vanillafullstrong/idempotent": True,
        "branching/vanillafullstrong/priority": 666666,  # use vanillafullstrong (highest priority)
    },
)

<br>

A simple node priority queue

In [ ]:
from itertools import chain

from heapq import heappush, heappop
from collections import namedtuple

NT = namedtuple("NT", "v,j")

def enqueue(m: scip.Model, pq: list, inq: set) -> None:
    # add leaves, childrean and siblings to the LP bound prio queue
    for node in chain(*m.getOpenNodes()):
        id = node.getNumber()

        # it is the set iff it is in the queue
        if id not in inq:
            heappush(pq, NT(node.getLowerbound(), id))
            inq.add(id)

An example

In [ ]:
p = next(it)

In [ ]:
# actset is a little redundant: we could just as well have
#  generated the same lists in python
pq, inq, trace = [], set(), []
obs, actset, rew, fin, nfo = env.reset(p)

m = env.model.as_pyscipopt()
enqueue(m, pq, inq)

sign = -1. if m.getObjectiveSense()[:3] == "max" else +1.
while actset is not None and True:
    val, node = heappop(pq)
    inq.remove(node)
    trace.append((node, sign * val, m.getPrimalbound()))

    leaves, children, siblings = actset
    print(leaves, children, siblings)

#     node_id = next(chain(leaves, children, siblings))
    obs, actset, rew, fin, nfo = env.step(node)

    m = env.model.as_pyscipopt()
    enqueue(m, pq, inq)
    if len(trace) > 0:
        break

In [ ]:
# Khalil
# set_objective_function_coefficient(out, col);
# set_number_constraints(out, col);
# set_static_stats_for_constraint_degree(out, rows);
# set_stats_for_constraint_positive_coefficients(out, coefficients);
# set_stats_for_constraint_negative_coefficients(out, coefficients);

In [ ]:
# `model.getState` is missing from scip-8.0
cands, *_ = m.getPseudoBranchCands()

In [ ]:
import networkx as nx

cols = m.getLPColsData()
rows = m.getLPRowsData()
# cons = m.getConss()
for col in cols:
    col.getObjCoeff()
    var_ = col.getVar()
    # var_.getIndex()
    # col_ = var_.getCol()
    break

# reconstruct the bi-partite graph for the current LP
# LP rows and cols are -ve and +ve nodes, respectively.
G = nx.DiGraph()
for row in rows:
    row_ = row.getLPPos()
    if row.getLPPos() == -1:
        continue

    # get the variables and the coefficient
    for col_, coef_ in zip(row.getCols(), row.getVals()):
        var_ = col_.getVar()
        G.add_edge(-(row_ + 1), var_.getIndex(), weight=coef_)
    # row.getLPPos()
    # row.getLhs(), row.getRhs()
#     break

In [ ]:
one, *nodes = chain(*m.getOpenNodes())

In [ ]:
cur = m.getCurrentNode()
cur.

In [ ]:
m.getRowDualSol(rows[2])

In [ ]:
m.getDualSolVal(cons[2])

In [ ]:
m.

In [ ]:
m.getLPColsData()  
# SCIPcolGetObj
ex = {
    t.vartuple[0].getIndex(): c
    for t, c in m.getObjective().terms.items()
}

v, *rest = m.getVars()

ex[v.getIndex()]

In [ ]:
n, d, p = zip(*trace)

from matplotlib import pyplot as plt
plt.plot(p[1:])
plt.plot(d[1:])

In [ ]:
env.model.dual_bound, env.model.primal_bound

In [ ]:
n: scip.Node = m.getCurrentNode()
n.getLowerbound()

In [ ]:
nodes = list(chain(*m.getOpenNodes()))

In [ ]:
n = nodes[4]
d = n.getDomchg()
d.getBoundchgs()

In [ ]:
n = m.getCurrentNode()
r, *rest = m.getLPRowsData()

In [ ]:
d = n.getDomchg()
b, *rest = d.getBoundchgs()
v = b.getVar()
c = v.getCol()
c.getLPPos()

In [ ]:
rest

In [ ]:
r.getLPPos()

<br>

In [ ]:
m = env.model.as_pyscipopt()

leaves, children, siblings = m.getOpenNodes()

In [ ]:
s_leaves = set(map(scip.Node.getNumber, leaves))
s_children = set(map(scip.Node.getNumber, children))
s_siblings = set(map(scip.Node.getNumber, siblings))

In [ ]:
s_leaves, s_siblings, s_children,

In [ ]:
m.getCurrentNode()

In [ ]:
from itertools import chain

node, *ignore = chain(*m.getOpenNodes())

In [ ]:
node.getNumber()

In [ ]:
env.step(1)